In [2]:
!wget https://www.kaggle.com/competitions/amex-default-prediction/data?select=train_data.csv

--2022-08-13 08:13:19--  https://www.kaggle.com/competitions/amex-default-prediction/data?select=train_data.csv
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘data?select=train_data.csv.1’

     0K .......... .......... .......... .......... ........    300K=0.2s

2022-08-13 08:13:20 (300 KB/s) - ‘data?select=train_data.csv.1’ saved [50064]



In [97]:
%pip install catboost -U

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 76.6 MB 39 kB/s             
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [31]:
import numpy as np # linear algebra
import pandas as pd
import joblib
import datetime
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
import copy
import gc

from scipy.sparse import csr_matrix
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import os
import time

# Essential DS libraries
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import torch
from catboost import CatBoostClassifier
from catboost import Pool, cv
from sklearn.preprocessing import LabelEncoder
import tqdm

from sklearn.linear_model import LogisticRegression
import torch.nn.functional as F
import math

import torch
import pandas as pd
import torch.nn as nn
from torch.autograd import Variable
#from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
import random
import torch
import pandas as pd
import torch.nn as nn
from torch.autograd import Variable
#from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
import random
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)
rng = np.random.default_rng(seed=42)

In [22]:
!unzip 'train_labels.csv.zip'

Archive:  train_labels.csv.zip
  inflating: train_labels.csv        


In [14]:
#!c1.8
df=read_csv("train_data.csv")

NameError: name 'read_csv' is not defined

In [16]:
#!c1.8
import pandas as pd

In [18]:
#!c1.8
df=pd.read_csv("train_data.csv")

In [25]:
#!c1.8
df.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,...,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,...,NaN,NaN,0.003954,0.003167,0.005032,NaN,0.009576,0.005492,0.009217,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,...,NaN,NaN,0.003269,0.007329,0.000427,NaN,0.003429,0.006986,0.002603,0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,...,NaN,NaN,0.006117,0.004516,0.003200,NaN,0.008419,0.006527,0.009600,0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,...,NaN,NaN,0.003671,0.004946,0.008889,NaN,0.001670,0.008126,0.009827,0


In [21]:
#!c1.8
len(df)

5531451

In [23]:
#!c1.8
labels=pd.read_csv('train_labels.csv')

In [24]:
#!c1.8
df=df.merge(labels, how='left',left_on='customer_ID',right_on='customer_ID')

In [26]:
#!c1.8

df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5531451 entries, 0 to 5531450
Columns: 191 entries, customer_ID to target
dtypes: float64(185), int64(2), object(4)
memory usage: 7.9+ GB


In [27]:
#!c1.8
all_cols = df.columns.to_list()
cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

dtype = {col: 'float16' for col in all_cols if col not in (cat_cols + ['customer_ID', 'S_2'])}

for col in cat_cols + ['customer_ID']:
    dtype[col] = 'category'
    


In [32]:
#!c1.8
for i in dtype.keys():
    if dtype[i]=='float16':
        df[i]=np.float16(df[i])

In [33]:
#!c1.8
gc.collect()

3

In [34]:
#!c1.8
joblib.dump(df,'small_df.sav')

['small_df.sav']

In [49]:
#!c1.8
CAT_COLUMNS = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

NUMERIC_COLUMNS = [ i for i in dtype.keys() if dtype[i]=='float16']
NUMERIC_COLUMNS.remove('target')
catboost_features=CAT_COLUMNS+NUMERIC_COLUMNS

In [102]:
#!c1.8
train.fillna(-1)

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
650470,1e4a396a77c9f3bc595809716f0570cd4be39685734f48...,2017-08-31,0.643555,0.270996,0.011101,1.007812,0.008347,0.126465,0.122009,0.006149,...,-1.0,-1.0,0.009758,0.008209,0.008629,-1.000000,0.000362,0.003223,0.001394,0.0
3689326,aa724b2873dadc69e4c02d4d01bdc7573311c3d2caee6f...,2018-03-24,0.543945,0.355713,0.443359,0.025558,0.003366,0.216064,0.006668,0.739258,...,-1.0,-1.0,0.005718,0.004761,0.007889,-1.000000,0.000877,0.000265,0.000416,1.0
5011625,e7d4c5ef7004b988f21985a290a8eb97f02c4d7c492328...,2017-09-07,0.894531,0.416504,0.033051,1.000977,0.007790,0.150391,0.002085,0.006393,...,-1.0,-1.0,0.002041,0.007996,0.002890,-1.000000,0.005955,0.004543,0.008514,0.0
3241525,95bd90a0547f8f42a093a0ea442b920eca9bef9e514ce5...,2017-08-14,0.772461,0.003546,0.025269,0.813965,0.005962,0.057709,0.004494,0.011116,...,-1.0,-1.0,0.005867,0.007202,0.005505,-1.000000,0.007412,0.005386,0.004082,0.0
1552740,47858837194d4c527d2e78b7797fd50f0c534cad7911ef...,2017-05-14,0.672852,0.183716,0.041779,1.003906,0.001216,0.350342,0.009979,0.008041,...,-1.0,-1.0,0.003824,0.007252,0.004658,-1.000000,0.009514,0.001801,0.006798,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2261263,687c56c38126587e4ee0909b6e232657eabf89d07bd388...,2017-11-18,0.426514,0.002314,0.005577,0.815918,0.003101,-1.000000,0.003685,0.000048,...,-1.0,-1.0,1.009766,0.000230,0.867676,0.039795,1.004883,0.004978,0.098755,0.0
3688365,aa67270ff22d3c6ee4bab735d3fe9298534b6f0d3ad0a7...,2017-06-08,0.486084,0.008377,0.003860,0.810059,0.008347,-1.000000,0.000150,0.003393,...,-1.0,-1.0,0.007183,0.009125,0.008011,-1.000000,0.008774,0.008247,0.002357,0.0
1728331,4fc1f1d5b1ed9f35a5b1871a751daaa247778a9eccf562...,2017-10-07,0.223022,0.005001,0.110107,0.485352,1.003906,0.951660,0.009895,0.022903,...,-1.0,-1.0,0.007763,0.002855,0.004528,-1.000000,0.008896,0.004955,0.008965,1.0
1905530,57eb0de17baefbdf95e630d6f81358b2290335138e96aa...,2017-07-07,0.854004,0.271729,1.101562,1.000977,0.007046,0.075867,0.007343,0.001590,...,-1.0,-1.0,0.008957,0.002771,0.002815,-1.000000,0.000754,0.002644,0.005783,0.0


In [43]:
#!c1.8
train,val  = train_test_split(df,test_size=0.2,stratify=df['target'])

In [123]:
#!c1.8
for i in range(len(CAT_COLUMNS)):
    train[CAT_COLUMNS[i]]=[str(j)  for j in train[CAT_COLUMNS[i]]]
    val[CAT_COLUMNS[i]]=[str(j)  for j in val[CAT_COLUMNS[i]]]

In [106]:
#!c1.8
for i in [ ]:
    train[CAT_COLUMNS[i]]=[(np.int(i) if not np.isnan(i) else 'nan' ) for i in train[CAT_COLUMNS[i]]]
    val[CAT_COLUMNS[i]]=[(np.int(i) if not np.isnan(i) else 'nan' ) for i in val[CAT_COLUMNS[i]]]

TypeError: list indices must be integers or slices, not str

In [107]:
#!c1.8

train['target']=[(np.int(i) if not np.isnan(i) else 'nan' ) for i in train['target']]
val['target']=[(np.int(i) if not np.isnan(i) else 'nan' ) for i in val['target']]

In [105]:
#!c1.8
train['target']=np.int(train['target'])

TypeError: cannot convert the series to <class 'int'>

In [119]:
#!c1.8
for i in CAT_COLUMNS:
    print(train[i].value_counts())

0      3768367
1       611449
2        43735
nan       1609
Name: B_30, dtype: int64
2      1562364
3      1004100
1       928599
5       355740
4       236156
7       207075
6       129517
nan       1609
Name: B_38, dtype: int64
1      2653304
0      1630263
nan     141593
Name: D_114, dtype: int64
0      4278286
nan     141593
1         5281
Name: D_116, dtype: int64
-1     1164728
3       933127
4       911182
2       533145
5       367937
6       275347
nan     141593
1        98101
Name: D_117, dtype: int64
0      3783944
1       499623
nan     141593
Name: D_120, dtype: int64
1      3409824
0       713231
-1      208548
nan      93557
Name: D_126, dtype: int64
CO    3295591
CR     744276
CL     350676
XZ      20590
XM       8476
XL       5551
Name: D_63, dtype: int64
O     2330489
U     1218227
R      672386
-1      29803
Name: D_64, dtype: int64
nan    3926437
1       493665
0         5058
Name: D_66, dtype: int64
6      2226571
5       960760
3       387701
4       381559
2    

In [98]:
#!c1.8
model = CatBoostClassifier(random_seed=42,
                           iterations=10000,
                           learning_rate=0.02,
                           l2_leaf_reg=3,
                           depth=5,
                           #task_type="GPU", # раскомментить, когда с GPU
                           thread_count=-1,
                           use_best_model=True,
                           leaf_estimation_method='Newton',
                           custom_loss=[ 'AUC'],
                           nan_mode='Min'

                          )



In [ ]:
#!c1.8
model.fit(train[catboost_features], train['target'],
                    cat_features = CAT_COLUMNS,
                    eval_set=(val[catboost_features], val['target']),
                    verbose=500, plot=False, early_stopping_rounds=600
         )

print(model.get_best_score())

0:	learn: 0.6726585	test: 0.6726691	best: 0.6726691 (0)	total: 3.39s	remaining: 9h 25m 3s


In [104]:
#!c1.8
train['target'].value_counts()

0.0    3322865
1.0    1102295
Name: target, dtype: int64

In [ ]:
#!c1.8


In [ ]:
#!c1.8
train.head()

In [ ]:
#!c1.8
